In [ ]:
!git clone https://github.com/facebookresearch/dinov2.git
%pip install -q -r /content/dinov2/requirements.txt
%pip install Pillow
%pip install chromadb

Cloning into 'dinov2'...
remote: Enumerating objects: 432, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 432 (delta 169), reused 150 (delta 135), pack-reused 157
Receiving objects: 100% (432/432), 1.46 MiB | 19.61 MiB/s, done.
Resolving deltas: 100% (210/210), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 962.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
import torch
import torchvision.transforms as transforms
import requests
import numpy as np
from tqdm import tqdm

In [ ]:
resize = [transforms.Resize((224, 224)), transforms.ToTensor()]
transformation = transforms.Compose(resize)

In [ ]:
image = requests.get('https://i1.sndcdn.com/artworks-SwoYPxk4GdfVzdkD-r6bHlg-t500x500.jpg')

In [ ]:
import requests
from PIL import Image
from io import BytesIO

url = 'https://i1.sndcdn.com/artworks-SwoYPxk4GdfVzdkD-r6bHlg-t500x500.jpg'
response = requests.get(url)
img = Image.open(BytesIO(response.content))

In [ ]:
dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitb14_pretrain.pth
100%|██████████| 330M/330M [00:01<00:00, 201MB/s]


In [ ]:
dinov2_vitb14.to('cuda')

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [ ]:
test = transformation(img).to('cuda')

In [ ]:
torch.unsqueeze(test, 0).shape

torch.Size([1, 3, 224, 224])

In [ ]:
from glob import glob

In [ ]:
paths = glob('/content/drive/MyDrive/Ran Frames/*.jpg')

In [ ]:
def embed(image_path):
  img = Image.open(image_path)
  img = transformation(img).to('cuda')
  embedding = dinov2_vitb14.forward(torch.unsqueeze(img, 0)).numpy(force=True)
  return embedding

In [ ]:
x = embed(paths[0])

In [ ]:
embeddings = {}
for path in tqdm(paths):
  embeddings[path] = embed(path)

100%|██████████| 2410/2410 [18:24<00:00,  2.18it/s]


In [ ]:
embeddings_list = {k: value.tolist()[0] for k, value in embeddings.items()}

In [ ]:
embeddings_list[paths[0]]

[0.6725409030914307,
 -0.34289854764938354,
 -1.2430464029312134,
 3.6684730052948,
 2.6642746925354004,
 -0.6323836445808411,
 1.3942476511001587,
 1.2019447088241577,
 -1.946792721748352,
 0.7500410079956055,
 3.475724697113037,
 -1.0767669677734375,
 -0.3611927628517151,
 2.1820268630981445,
 2.4240546226501465,
 -1.8094121217727661,
 1.5991854667663574,
 -2.684476375579834,
 0.4463218152523041,
 -2.1850762367248535,
 0.5669233202934265,
 0.8684695959091187,
 -0.08781882375478745,
 -0.7754220366477966,
 -1.5233047008514404,
 -0.5899635553359985,
 0.3953021168708801,
 -0.8099638223648071,
 1.1303112506866455,
 3.0384721755981445,
 -0.030873984098434448,
 2.56400465965271,
 -0.5448119044303894,
 0.20159907639026642,
 1.286526083946228,
 0.8626807332038879,
 -1.4721417427062988,
 -0.7996973395347595,
 0.4114685356616974,
 2.090998649597168,
 1.5454497337341309,
 -0.09502628445625305,
 0.9610675573348999,
 -1.6001907587051392,
 1.519319772720337,
 1.0478605031967163,
 -3.074756860733032

In [ ]:
json.dump(embeddings_list, open('embeddings.json', 'w'))